In [ ]:
# Imports

import tensorflow as tf 
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np 

import matplotlib.pyplot as plt 


In [3]:
# Import Images

IMG_SIZE = 254
BATCH_SIZE = 32

train_path = "DataSplit/train"
validation_path = "DataSplit/val"
test_path = "DataSplit/test"

img_rescale = ImageDataGenerator(
    rescale = 1./255
)

train_data_augmentation = ImageDataGenerator( # Define params for data augmentation. Reduces overfitting 
    rescale = 1./255,
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True
)

train_gen = train_data_augmentation.flow_from_directory(
    train_path, 
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "binary"
)

val_gen = img_rescale.flow_from_directory(
    validation_path, 
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "binary"
)

test_gen = img_rescale.flow_from_directory(
    test_path, 
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = "binary"
)

Found 9665 images belonging to 2 classes.
Found 2070 images belonging to 2 classes.
Found 2073 images belonging to 2 classes.


In [4]:
# Create CNN Model

model = models.Sequential([

    # Input Layer
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2 ,2), 

    # Hidden Layer 1
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2 ,2), 

    # Hidden Layer 2
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    # Output Layer
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'), # Binary output 
])

# Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

/Users/taniarizwan/Desktop/CovidPrediction/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 252, 252, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     7,372,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,466,177 (28.48 MB)

 Trainable params: 7,466,177 (28.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model

history = model.fit(
    train_gen,
    validation_data = val_gen,
    epochs = 10
)

Epoch 1/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: 0.7391 - loss: 0.5371 - val_accuracy: 0.7377 - val_loss: 0.4862
Epoch 2/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7506 - loss: 0.4987

In [ ]:
# Visualize performance

# Accuracy plot
plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.legend(), plt.title("Accuracy"), plt.show()

# Loss plot
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.legend(), plt.title("Loss"), plt.show()

In [ ]:
# Plot Confusion Matrix

y_true = test_gen.classes
y_pred_probabilities = model.predict(test_gen) # Let model predict on test data
y_pred = (y_pred_probabilities > 0.5).astype(int)

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=["Normal", "COVID"], yticklabels=["Normal", "COVID"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

print(classification_report(y_true, y_pred, target_names = ["Normal", "COVID"]))
